In [4]:
import torch
import torchvision
from torch import nn
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.utils import save_image
from torchvision.datasets import MNIST
import os

if not os.path.exists('./dc_img'):
    os.mkdir('./dc_img')


def to_img(x):
    x = 0.5 * (x + 1)
    x = x.clamp(0, 1)
    x = x.view(x.size(0), 1, 28, 28)
    return x


num_epochs = 100
batch_size = 128
learning_rate = 1e-3

img_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5, ))
])

dataset = MNIST('./data', transform=img_transform)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)


class autoencoder(nn.Module):
    def __init__(self):
        super(autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(1, 16, 3, stride=3, padding=1),  # b, 16, 10, 10
            nn.ReLU(True),
            nn.MaxPool2d(2, stride=2),  # b, 16, 5, 5
            nn.Conv2d(16, 8, 3, stride=2, padding=1),  # b, 8, 3, 3
            nn.ReLU(True),
            nn.MaxPool2d(2, stride=1)  # b, 8, 2, 2
        )
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(8, 16, 3, stride=2),  # b, 16, 5, 5
            nn.ReLU(True),
            nn.ConvTranspose2d(16, 8, 5, stride=3, padding=1),  # b, 8, 15, 15
            nn.ReLU(True),
            nn.ConvTranspose2d(8, 1, 2, stride=2, padding=1),  # b, 1, 28, 28
            nn.Tanh()
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x


model = autoencoder()
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate,
                             weight_decay=1e-5)

for epoch in range(num_epochs):
    for data in dataloader:
        img, _ = data
        img = Variable(img)
        # ===================forward=====================
        output = model(img)
        loss = criterion(output, img)
        # ===================backward====================
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    # ===================log========================
    print('epoch [{}/{}], loss:{:.4f}'
          .format(epoch+1, num_epochs, loss.item()))
    if epoch % 10 == 0:
        pic = to_img(output.cpu().data)
        save_image(pic, './dc_img/image_{}.png'.format(epoch))

torch.save(model.state_dict(), './conv_autoencoder.pth')

epoch [1/100], loss:0.2123
epoch [2/100], loss:0.1674
epoch [3/100], loss:0.1510
epoch [4/100], loss:0.1450
epoch [5/100], loss:0.1442
epoch [6/100], loss:0.1452
epoch [7/100], loss:0.1386
epoch [8/100], loss:0.1357
epoch [9/100], loss:0.1327
epoch [10/100], loss:0.1277
epoch [11/100], loss:0.1363
epoch [12/100], loss:0.1343
epoch [13/100], loss:0.1324
epoch [14/100], loss:0.1172
epoch [15/100], loss:0.1284
epoch [16/100], loss:0.1239
epoch [17/100], loss:0.1215
epoch [18/100], loss:0.1233
epoch [19/100], loss:0.1201
epoch [20/100], loss:0.1206
epoch [21/100], loss:0.1167
epoch [22/100], loss:0.1247
epoch [23/100], loss:0.1263
epoch [24/100], loss:0.1274
epoch [25/100], loss:0.1206
epoch [26/100], loss:0.1250
epoch [27/100], loss:0.1249
epoch [28/100], loss:0.1174
epoch [29/100], loss:0.1149
epoch [30/100], loss:0.1250
epoch [31/100], loss:0.1235
epoch [32/100], loss:0.1225
epoch [33/100], loss:0.1158
epoch [34/100], loss:0.1277
epoch [35/100], loss:0.1208
epoch [36/100], loss:0.1221
e